# Example on loading probabilities output from the cram file

For installation use the following pip install, required python 3.11



In [120]:
# pip install ugbio_core[vcfbed]

In [121]:
import pysam
import numpy as np

In [122]:
from util import Flow_prob_from_sam_record, extract_all_tags_to_dataframe
from util import key2seq

## Load cram file 

This load the sam file and pull the probabilites from it and store to disk.

It takes ~2min for the example ~200k sam file

In [123]:
# !aws s3 cp s3://ultimagen-workflow-resources-us-east-1/test_data/cram_example/cram_example.cram /data/Runs/examples/cram_example.cram

In [124]:
cram_file = '/data/Runs/examples/cram_example.cram'

In [125]:
alignment_file = pysam.AlignmentFile(cram_file, 'r')
Flow_prob_from_sam_record(alignment_file, batch_size=50_000, output_file = '/data/Runs/examples/run_name.flow_prob', max_hmer_size=20, max_len=500)

[E::cram_index_load] Could not retrieve index file for '/data/Runs/examples/cram_example.cram'


Processed 50000 records, in batch 0
Processed 100000 records, in batch 1
Processed 150000 records, in batch 2
Processed 200000 records, in batch 3
Processed 238479 records, in batch 4
Finished processing 238479 records


## Validate the data created

load directly the cram file

In [126]:
df = extract_all_tags_to_dataframe(cram_file)

In [127]:
print('there are',len(df),'reads in the sam file')

there are 238479 reads in the sam file


### sequence of sample read

In [128]:
df.head()

,QNAME,FLAG,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,...,SA,RG,tm,a3,XA,XB,MI,DS,Index,OP
0,030912011001_2-0000020911,16,chr1,10173,0,151S60M,*,0,0,CTACTATCTACTACTACATACTACTACTACTACTACTACTACTACT...,...,"chr15,61595589,+,131S59M21S,1,0;chr9,19592496,...",Z0115,NaN,NaN,NaN,NaN,NaN,NaN,20911,2
1,030912011001_2-0000098204,0,chr1,20114,0,335M,*,0,0,ACTTAGAAAAGGCCGCGGTGAGTCCCAGGGGCCAGCACTGCTCGAA...,...,NaN,Z0115,NaN,NaN,NaN,NaN,NaN,NaN,98204,2
2,030912011001_2-0000040692,0,chr1,24403,0,22M2D74M1D53M1I32M1I12M1D26M,*,0,0,CCTTGCACACGAGCACTGCTGGTAATATTTGTTGGCTGCAGGAAAA...,...,NaN,Z0115,Q,NaN,NaN,NaN,NaN,NaN,40692,2
3,030912011001_1-0000094689,0,chr1,26444,0,289M,*,0,0,TTTCTTGTTAGTGTGTGTGTGTTTGCTCACACATATGCGTGAAAGA...,...,NaN,Z0115,A,290.0,NaN,NaN,NaN,NaN,94689,1
4,030912011001_1-0000015469,16,chr1,65740,25,281M,*,0,0,TAGAGAAATTAATATGAATAATGTTAGCAAGAATAACCCTTGTTAT...,...,NaN,Z0115,A,282.0,NaN,NaN,NaN,NaN,15469,1


In [129]:
i = 1

print('sequence of the first 100 bases of read number ', i)
print( df['SEQ'][i][:100] )

sequence of the first 100 bases of read number  1
ACTTAGAAAAGGCCGCGGTGAGTCCCAGGGGCCAGCACTGCTCGAAATGTACAGCATTTCTCTTTGTAACAGGATTATTAGCCTGCTGTGCCCGGGGAAA


### load probability vector of the

In [130]:
prob = np.memmap('/data/Runs/examples/run_name.flow_prob', dtype=np.float32).reshape(-1, 500, 21)

probability_flow_i = prob[i].copy()

### from probabilities to hmers

In [131]:
called_hmers_from_sample = probability_flow_i.argmax(axis=-1)

### from hmers to sequence:


In [132]:
sequence = key2seq(called_hmers_from_sample, flow_order=None, start=0)

print('sequence of the first 100 bases of read number ', i)
print( sequence[:100] )

sequence of the first 100 bases of read number  1
CTTAGAAAAGGCCGCGGTGAGTCCCAGGGGCCAGCACTGCTCGAAATGTACAGCATTTCTCTTTGTAACAGGATTATTAGCCTGCTGTGCCCGGGGAAAA


There is some issue typical to the first cycle in the transformation, causes here to miss the correct probability in the first cycle.

After the first cycle it matches

In [133]:
print('are the first 200 bases of the sequence the same as the SEQ tag in the sam file?')

sequence[:200]==df['SEQ'][i][1:201]

are the first 200 bases of the sequence the same as the SEQ tag in the sam file?


True